In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('movies_metadata.csv', encoding='utf-8')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
# "장르" column 정리
genres = []
for list_ in data["genres"].apply(lambda x: eval(x)):
    genres.extend(list_) if len(list_) > 0 else genres.append(np.nan)
    
# Unique Genre-Id Tuple
genres_unique = [set(tuple(x.items()) if type(x) == dict else None for x in genres)]

In [3]:
# retain genre by name

genre_byname = data["genres"].apply(lambda x: [*map(lambda y: y["name"], eval(x))] if len(eval(x)) > 0 else np.nan)

In [4]:
# generate new_data with necessary data

new_data = pd.DataFrame(columns=["original_title","overview","genres"])

for i in new_data.columns:
  new_data[i] = data[i].copy()

new_data["genres"] = genre_byname

new_data = new_data.dropna().reset_index(drop=True)

In [5]:
genre_counts = {}
for genres in new_data['genres']:
  for genre in genres:
    if genre not in genre_counts:
        genre_counts[genre] = 1
    else:
        genre_counts[genre] += 1
genre_counts

{'Animation': 1920,
 'Comedy': 12806,
 'Family': 2732,
 'Adventure': 3470,
 'Fantasy': 2290,
 'Romance': 6673,
 'Drama': 20023,
 'Action': 6565,
 'Crime': 4269,
 'Thriller': 7586,
 'Horror': 4660,
 'History': 1379,
 'Science Fiction': 3028,
 'Mystery': 2451,
 'War': 1310,
 'Foreign': 1599,
 'Music': 1588,
 'Documentary': 3886,
 'Western': 1035,
 'TV Movie': 751,
 'Carousel Productions': 1,
 'Vision View Entertainment': 1,
 'Telescene Film Group Productions': 1,
 'Aniplex': 1,
 'GoHands': 1,
 'BROSTA TV': 1,
 'Mardock Scramble Production Committee': 1,
 'Sentai Filmworks': 1,
 'Odyssey Media': 1,
 'Pulser Productions': 1,
 'Rogue State': 1,
 'The Cartel': 1}

In [6]:
# exclude insignificant genres
exclude_list = []
for gen in genre_counts:
  if genre_counts[gen] == 1:
    exclude_list.append(gen)
exclude_list.extend(["TV Movie","Foreign", "Animation"])
for exc in exclude_list:
  del genre_counts[exc]

In [ ]:
# draw the correlation plot
# make the vectors of each genre

data_nn = new_data.copy()
for gen in genre_counts.keys():
    data_nn[gen] = [0] * len(data_nn)

n = 0
for gen in genre_counts.keys():
    for i in data_nn['genres']:
        if gen in i:
            data_nn.loc[n,[gen]] = 1
            #data_nn[gen][n] = 1
        n += 1
    print(n == len(data_nn))
    n = 0

In [ ]:
data_corr = data_nn.drop(['original_title','overview','genres'], axis=1)
corr = data_corr.corr()
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15))
sns.heatmap(round(corr,2), annot=True, linewidths=2)

In [7]:
# sort the genres by the counts
# exclude all the unnecessary genres except for the top 5

genres_sorted = [j[0] for j in sorted(genre_counts.items(), key = lambda x: x[1])]
genres_sorted
exclude_list.extend(genres_sorted[:-5])

In [8]:
# keep documentary and horror

exclude_list.remove("Documentary")
exclude_list.remove("Horror")

In [9]:
# keep only the selected 7 genres

genres_excluded = []
for i in new_data["genres"]:
    new = list(set(i) - set(exclude_list))
    genres_excluded.append(new if len(new) > 0 else np.nan)

In [10]:
# replace the genres colum with only the necessary information
# drop the rows with no genre remaining

new_data["genres"] = genres_excluded
new_data = new_data.dropna().reset_index(drop=True)

In [11]:
# combine Drama and Romance into Drama/Romance

def combine_rd(genres):
    l = genres
    for i in range(len(genres)):
        if l[i] == 'Romance' or l[i] == 'Drama':
            l[i] = 'Drama/Romance'
    new_genres = list(dict.fromkeys(l))
    return new_genres

In [12]:
new_data['genres'] = new_data['genres'].apply(lambda x: combine_rd(x))

In [15]:
# count genres

new_genre_counts = {}
for genres in new_data['genres']:
  for genre in genres:
    if genre not in new_genre_counts:
        new_genre_counts[genre] = 1
    else:
        new_genre_counts[genre] += 1
new_genre_counts

{'Comedy': 12806,
 'Drama/Romance': 22241,
 'Action': 6565,
 'Thriller': 7586,
 'Horror': 4660,
 'Documentary': 3886}

In [16]:
# sort genres in order

new_genres_sorted = [j[0] for j in sorted(new_genre_counts.items(), key = lambda x: x[1])]
new_genres_sorted

['Documentary', 'Horror', 'Action', 'Thriller', 'Comedy', 'Drama/Romance']

In [17]:
# leave every row with only 1 genre
# if a row has multiple genres, the genre with the least count survives

new_data['one_genre'] = [0] * len(new_data)
for e, i in enumerate(new_data['genres']):
    flag = True
    for genre in new_genres_sorted:
        if flag:
            if genre in i:
                new_data.loc[e,['one_genre']] = genre
                flag = False
    if e % 1000 == 0:
        print(e,', ',new_data.iloc[e]['one_genre'])

0 ,  Comedy
1000 ,  Horror
2000 ,  Drama/Romance
3000 ,  Action
4000 ,  Horror
5000 ,  Comedy
6000 ,  Action
7000 ,  Drama/Romance
8000 ,  Comedy
9000 ,  Thriller
10000 ,  Action
11000 ,  Comedy
12000 ,  Action
13000 ,  Comedy
14000 ,  Thriller
15000 ,  Horror
16000 ,  Drama/Romance
17000 ,  Action
18000 ,  Action
19000 ,  Horror
20000 ,  Comedy
21000 ,  Horror
22000 ,  Drama/Romance
23000 ,  Comedy
24000 ,  Comedy
25000 ,  Thriller
26000 ,  Drama/Romance
27000 ,  Drama/Romance
28000 ,  Horror
29000 ,  Action
30000 ,  Comedy
31000 ,  Drama/Romance
32000 ,  Drama/Romance
33000 ,  Drama/Romance
34000 ,  Comedy
35000 ,  Comedy
36000 ,  Horror
37000 ,  Comedy
38000 ,  Comedy
39000 ,  Comedy


In [18]:
new_data.head()

,original_title,overview,genres,one_genre
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",[Comedy],Comedy
1,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Comedy, Drama/Romance]",Comedy
2,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama/Romance]",Comedy
3,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],Comedy
4,Heat,"Obsessive master thief, Neil McCauley leads a ...","[Action, Drama/Romance, Thriller]",Action


In [19]:
# make an index number for each genre

genre_idx = new_data['one_genre'].copy()
for index, genre in enumerate(new_genres_sorted):
  genre_idx[genre_idx == genre] = index

In [20]:
new_data['genre_idx'] = genre_idx

In [25]:
new_data.head()

,original_title,overview,genres,one_genre,genre_idx
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",[Comedy],Comedy,4
1,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Comedy, Drama/Romance]",Comedy,4
2,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama/Romance]",Comedy,4
3,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],Comedy,4
4,Heat,"Obsessive master thief, Neil McCauley leads a ...","[Action, Drama/Romance, Thriller]",Action,2


In [26]:
new_data['one_genre'].value_counts()

Drama/Romance    11232
Comedy           10463
Action            6048
Horror            4632
Documentary       3886
Thriller          3495
Name: one_genre, dtype: int64

In [31]:
# remove unnecessary text in 'overview' column

import re

row_drop = []
for e,i in enumerate(new_data["overview"]):
    new_data["overview"][e] = re.sub('(https?://)[^ ]+','', i)
    new_data["overview"][e] = re.sub('[^ ]+(www.)+[^ ]+','', i)
    new_data["overview"][e] = re.sub('hancinema.net[^ ]+','', i)
    new_data["overview"][e] = re.sub('(mc)[^ ]+','',i)
    if "No overview" in i or "No Overview" in i or "no overview" in i:
        row_drop.append(e)
        
new_data = new_data.drop(row_drop).reset_index(drop=True)

In [34]:
new_data.head()

,original_title,overview,genres,one_genre,genre_idx
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",[Comedy],Comedy,4
1,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Comedy, Drama/Romance]",Comedy,4
2,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama/Romance]",Comedy,4
3,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],Comedy,4
4,Heat,"Obsessive master thief, Neil McCauley leads a ...","[Action, Drama/Romance, Thriller]",Action,2


In [35]:
final_data = new_data[['original_title', 'overview', 'one_genre', 'genre_idx']]

In [36]:
final_data

,original_title,overview,one_genre,genre_idx
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Comedy,4
1,Grumpier Old Men,A family wedding reignites the ancient feud be...,Comedy,4
2,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy,4
3,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,4
4,Heat,"Obsessive master thief, Neil McCauley leads a ...",Action,2
...,...,...,...,...
39629,The Burkittsville 7,A film archivist revisits the story of Rustin ...,Horror,1
39630,Robin Hood,"Yet another version of the classic epic, with ...",Action,2
39631,رگ خواب,Rising and falling between a man and woman.,Drama/Romance,5
39632,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,Drama/Romance,5


In [37]:
final_data.to_csv('movie_train_data.csv', index=False)